In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.svm import SVR

# Load your NDVI time series data into a Pandas DataFrame
# Replace 'your_data.csv' with the actual file name or data source
stock_df = pd.read_csv('Dengue Dataset 2022.csv')
stock_df['date']=pd.to_datetime(stock_df['date'], format='%d/%m/%Y')

# Extract NDVI values as a NumPy array
ndvi_data = stock_df['date'].values

# Normalize the data using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
ndvi_data = scaler.fit_transform(ndvi_data.reshape(-1, 1))

# Define the sequence length (number of time steps) and split the data into input and target sequences
sequence_length = 10  # You can adjust this to your needs
X, y = [], []
for i in range(len(ndvi_data) - sequence_length):
    X.append(ndvi_data[i:i+sequence_length])
    y.append(ndvi_data[i+sequence_length])

X, y = np.array(X), np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an LSTM model for feature extraction
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(sequence_length, 1)))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32)

# Extract features using the LSTM model
features_train = lstm_model.predict(X_train)
features_test = lstm_model.predict(X_test)

# Create an SVR model for time prediction
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.2)

# Train the SVR model using the extracted features
svr_model.fit(features_train, y_train)

# Use the SVR model to predict future time points
future_features = lstm_model.predict(X_test[-1].reshape(1, sequence_length, 1))  # Use the last observed data as input
future_ndvi = svr_model.predict(future_features)

# Inverse transform the prediction to the original scale
future_ndvi = scaler.inverse_transform(future_ndvi.reshape(-1, 1))

print("Predicted NDVI for the future time point:", future_ndvi[0, 0])

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 0.3757
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: 0.3580
Epoch 3/50
1/1 [==============================] - 0s 8ms/step - loss: 0.3405
Epoch 4/50
1/1 [==============================] - 0s 8ms/step - loss: 0.3232
Epoch 5/50
1/1 [==============================] - 0s 8ms/step - loss: 0.3060
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: 0.2894
Epoch 7/50
1/1 [==============================] - 0s 6ms/step - loss: 0.2731
Epoch 8/50
1/1 [==============================] - 0s 12ms/step - loss: 0.2569
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: 0.2408
Epoch 10/50
1/1 [==============================] - 0s 9ms/step - loss: 0.2249
Epoch 11/50
1/1 [==============================] - 0s 9ms/step - loss: 0.2090
Epoch 12/50
1/1 [==============================] - 0s 10ms/step - loss: 0.1932
Epoch 13/50
1/1 [==============================] - 0s 9ms/step - loss: 0

e:\Python\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
# Plot the results
plt.figure(figsize=(20, 10))
plt.plot(y_test, label='True NDVI')
plt.plot(y_pred, label='Predicted NDVI')
plt.legend()
plt.title('NDVI Prediction with LSTM and SVR')
plt.show()

NameError: name 'plt' is not defined